<a href="https://colab.research.google.com/github/kiryu-3/Prmn2023/blob/main/Python/LINEBot/LINEBot_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python×LINEBot（応用編）

In [ ]:
# 最初に実行してください
!pip install line-bot-sdk
!pip install pyngrok
!pip install python-dotenv

## 初めに

- [LINE Developersコンソール](https://developers.line.biz/console/)にアクセスし、チャネルを作成してください。
手順については[LINE Developersコンソールでチャネルを作成する](https://developers.line.me/ja/docs/messaging-api/getting-started/)を参考にしてください。


  - チャネルの「チャネル基本設定」で、
チャネルアクセストークンとチャネルシークレットを取得してください。


  - 取得したチャネルアクセストークンとチャネルシークレットを、
"Your Channel Access Token"と"Your Channel Secret"の部分に置き換えてください。

  - WebHookURL を、 `Public URL: https://xx-xx-xx-xx-xx..ngrok.io`となっているところをふまえて
`https://xx-xx-xx-xx-xx..ngrok.io/callback` に変更してください。

![](https://imgur.com/x7YJM8b.png)

In [ ]:
# 自分のトークンを設定してください
import os

directory = 'content/info'
if not os.path.exists(directory):
    os.makedirs(directory)

file_path = os.path.join(directory, '.env')
with open(file_path, 'w') as f:
    f.write('CHANNEL_ACCESS_TOKEN = Your Channel Access Token\n')
    f.write('CHANNEL_SECRET = Your Channel Secret\n')

## 例1 OpenWeather APIの使用

OpenWeatherMapという気象情報を管理しているAPIを使うと、
 特定の地域の気象情報を取得することができます。

 OpenWeather APIキーの取得方法について、参考記事は[こちら](https://dev.classmethod.jp/articles/openweather_-pyowm/)です。

本資料では解説していないAPIについても解説があります。



APIはほかにもたくさんありますので、自分の実現したいAPIを適宜探し利用していただければと思います。

以下では、地域名を入力するとその地点の現在の天気を取得することができます。

＜実行イメージ＞

![](https://imgur.com/fmIGr0J.png)

In [ ]:
# .envファイルを上書きしてAPIキーを追加する

with open("./info/.env", "a") as f:
    f.write("\nAPI_KEY=YOUR_API_KEY\n")

In [ ]:
import os
from pathlib import Path
import uuid
import requests
import json

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])
api_key = os.getenv("API_KEY")

# カウント用のグローバル変数
not_tenki_mode = True

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

def get_weather(city):
    # OpenWeatherMap APIのベースURL
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    # APIリクエストに必要なパラメータ
    params = {
        "q": city,              # 指定した都市名
        "appid": api_key,       # APIキー（このコード内では定義されていないので追加する必要があります）
        "units": "metric"       # 温度の単位を摂氏に指定
    }

    # APIリクエストを送信してレスポンスを取得
    response = requests.get(base_url, params=params)

    # レスポンスデータをJSON形式に変換
    data = response.json()

    # レスポンスデータのステータスコードが200（成功）の場合
    if data["cod"] == 200:
        # 天気のマークと対応する絵文字の辞書
        weather_mark_list = {"Clear": "☀️", "Clouds": "🌥", "Rain": "☔️", "Mist": "🌫", "Fog": "🌫"}

        # 現在の天気のマーク（絵文字）
        weather_mark = weather_mark_list[data['weather'][0]['main']]

        # 現在の気温
        temperature = data["main"]["temp"]

        # 現在の湿度
        humidity = data["main"]["humidity"]

        # 天気の詳細な説明
        description = data["weather"][0]["description"]

        # 出力メッセージを作成
        output_message = f"現在の天気：{weather_mark}\n現在の気温：{temperature}°C\n現在の湿度：{humidity}%\n詳細：{description}"

        # 出力メッセージを返す
        return output_message
    else:
        # ステータスコードが200以外の場合はエラーメッセージを返す
        return "Error"

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global not_tenki_mode  # グローバル変数を参照するために宣言

    # テキストメッセージを返す関数
    def output_text(output_message):
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=output_message)
        )

    # ユーザーが入力したメッセージを変数に代入
    input_message = event.message.text

    # 無限ループ
    while True:
        # "天気"と入力された場合
        if input_message == "天気":
            # 天気モードではない場合
            if not_tenki_mode:
                # 天気モードのスタート
                reply_message = "気象情報を取得したい地域を入力してください\n例：札幌市"
                output_text(reply_message)
                not_tenki_mode = False
                break

            # すでに天気モードである場合
            else:
                # 天気モードの終了
                reply_message = "気象情報の取得を終了します"
                output_text(reply_message)
                not_tenki_mode = True
                break

        # "天気"以外が入力された場合
        else:
            # 天気モードではない場合
            if not_tenki_mode:
                # オウム返し
                reply_message = input_message
                output_text(reply_message)
                break

            else:
                # 気象情報の取得
                flag = get_weather(input_message)
                if flag == "Error":
                    output_message = "気象情報を取得できませんでした\n正しい地域を入力してください"
                    output_text(output_message)
                    break
                else:
                    output_message = flag
                    plus_message = "続けて地域を入力してください\n気象情報の取得を終了したい場合は「天気」と入力してください"
                    # 2つのメッセージを返信
                    line_bot_api.reply_message(
                        event.reply_token,
                        [TextSendMessage(text=output_message), TextSendMessage(text=plus_message)]
                    )
                    break

ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行


取得して得られたjsonデータは以下のような形式になります。

このjsonをPythonの辞書に変換して、欲しい情報だけを抽出し返します。

![](https://imgur.com/6kB4uXC.png)

## 例2 クイックリプライ

Botが送信したメッセージに、クイックリプライボタンを設定することで、
トーク画面の下部にボタンが表示されます。

ユーザーはボタンをタップするだけで、ボットからのメッセージに簡単に返信できます。

クイックリプライについては[公式のドキュメント：クイックリプライ](https://developers.line.biz/ja/docs/messaging-api/message-types/#quick-reply)などを参考にしてください。

作り方について、参考記事は[こちら](https://zanote.net/python/quick-reply/)です。

この記事ではクイックリプライを使ってじゃんけんbotを作っています。

以下では、クイックリプライを使って電卓プログラムを少し使いやすいものにしています。

In [ ]:
import os
from pathlib import Path
import uuid

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, PostbackEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage, QuickReply, QuickReplyButton, PostbackAction
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])

# カウント用のグローバル変数
session = {}  # 電卓セッションの情報を保持する辞書
not_dentaku = True  # 電卓モードかどうかを示すフラグ

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

# 計算結果を返す関数
def calculate_result():
    num1 = session['num1']
    operator = session['operator']
    num2 = session['num2']

    if operator == '+':
        result = num1 + num2
    elif operator == '-':
        result = num1 - num2
    elif operator == '*':
        result = num1 * num2
    elif operator == '/':
        result = num1 / num2
    else:
        result = None

    result = round(result, 2)
    return f"{num1} {operator} {num2} = {str(result)}"

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global session  # 入力情報を管理するグローバル変数
    global not_dentaku  # 電卓モードかどうかを管理するグローバル変数

    # テキストメッセージを返す関数
    def output_text(output_message):
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=output_message)
        )

    # ユーザーが入力したメッセージを変数に代入
    input_message = event.message.text
    # 無限ループ
    while True:
        # "電卓"と入力された場合
        if input_message == "電卓":
            # 電卓モードではない場合
            if not_dentaku:
                # 電卓モードのスタート
                reply_message = "電卓を開始しますか？"
                make_quick_reply(event.reply_token, reply_message, ("開始します！", "いいえ"))
                break

            # すでに電卓モードである場合
            else:
                # 電卓モードの終了
                reply_message = "電卓を終了しますか？"
                make_quick_reply(event.reply_token, reply_message, ("終了します", "いいえ"))
                break

        # "電卓"以外が入力された場合
        else:
            # 電卓モードではない場合
            if not_dentaku:
                # オウム返し
                reply_message = input_message
                output_text(reply_message)
                break

        # まだ一つ目の数が入力されていない場合
        if 'num1' not in session:
            # 数値かどうかを判断
            try:
                # 数値の場合は処理続行
                session['num1'] = float(input_message)
                reply_message = '演算子を選択してください'
                make_quick_reply(event.reply_token, reply_message, ("+", "-", "*", "/"))
                break
            except ValueError:
                # 数値ではない場合は再度入力させる
                reply_message = '無効な数値です\n最初の数値を入力してください'
                output_text(reply_message)
                break

        # # まだ演算子が入力されていない場合
        # if 'operator' not in session:
        #     # 演算子の場合は処理続行
        #     session['operator'] = input_message
        #     reply_message = '2番目の数値を入力してください'
        #     output_text(reply_message)
        #     break

        # まだ二つ目の数が入力されていない場合
        if 'num2' not in session:
            # ゼロ除算かどうかを判断
            if session['operator'] == "/" and input_message == "0":
                reply_message = '無効な数値です\n2番目の数値を入力してください'
                output_text(reply_message)
                break

            # 数値かどうかを判断
            try:
                # 数値の場合は処理続行
                session['num2'] = float(input_message)

                # これまでの入力データを使って計算
                session['kekka'] = calculate_result()

                # これまでの入力データを使って計算
                reply_message = calculate_result()

                # 電卓続行の確認
                make_quick_reply(event.reply_token, reply_message, ("電卓を続行します", "電卓を終了します"))

                # セッション情報をクリア
                session = {}
                break

            except ValueError:
                # 数値ではない場合は再度入力させる
                reply_message = '無効な数値です\n2番目の数値を入力してください'
                output_text(reply_message)
                break

# postback messageが返された時のアクション
@handler.add(PostbackEvent)
def on_postback(event):
    global session  # 入力情報を管理するグローバル変数
    global not_dentaku  # 電卓モードかどうかを管理するグローバル変数
    postback_msg = event.postback.data

    # テキストメッセージを返す関数
    def output_text(output_message):
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=output_message)
        )

    if postback_msg in ("+", "-", "*", "/"):
        # 演算子の場合は処理続行
        session['operator'] = postback_msg
        reply_message = '2番目の数値を入力してください'
        output_text(reply_message)

    elif postback_msg in ("開始します！", "いいえ"):
        if postback_msg == "開始します！":
            # 電卓モードのスタート
            reply_message = "電卓を開始します\n最初の数値を入力してください"
            output_text(reply_message)
            not_dentaku = False
        else:
            reply_message = "電卓"
            output_text(reply_message)

    elif postback_msg in ("終了します", "いいえ"):
        if postback_msg == "終了します":
            # 電卓モードの終了
            reply_message = "電卓を終了します"
            output_text(reply_message)
            not_dentaku = True
        else:
            reply_message = "電卓"
            output_text(reply_message)

    elif postback_msg in ("電卓を続行します", "電卓を終了します"):
        if postback_msg == "電卓を終了します":
            # 電卓モードの終了
            reply_message = "電卓を終了します"
            output_text(reply_message)
            not_dentaku = True
        else:
            # 電卓モードの終了
            reply_message = "電卓を続行します\n最初の数値を入力してください"
            output_text(reply_message)


def make_quick_reply(token, text, keys):
    items = []
    for key in keys:
      items.append(QuickReplyButton(action=PostbackAction(label=key, data=key)))
    messages = TextSendMessage(text=text,
                            quick_reply=QuickReply(items=items))
    line_bot_api.reply_message(token, messages=messages)

ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行

## 例3 Flexメッセージ



Flex Messageを使うことで、レイアウトを自由にカスタマイズしてメッセージを送信できます。

Flex Messageについては[公式のドキュメント：Flex Message](https://developers.line.biz/ja/docs/messaging-api/message-types/#flex-messages)などを参考にしてください。

作り方について、参考記事は[こちら](https://sajyounokousatu.hatenablog.com/entry/linebot-flexmessage)です。

[Flex Message Simulator](https://developers.line.biz/flex-simulator/?status=success&status=success)というものを使うと素早くレイアウトを作れるようです。

ここでは、"プロメン"と入力されたときに、プロメン2023の参考資料2つをFlex Messageで返すようにします。

＜実行イメージ＞

![](https://imgur.com/aowDQG8.png)

In [ ]:
import os
from pathlib import Path
from datetime import datetime
import pytz

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage, FlexSendMessage
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

def make_flex():
    temp = [
        {
          "type": "text",
          "text": "プロメン2023",
          "weight": "bold",
          "size": "xl"
        }
    ]

    # Flexの情報
    text_dict = {
        "プロメン資料群": "https://github.com/kiryu-3/Prmn2023",
        "プロメン参考教材群": "https://scrapbox.io/Prmn2023/%E3%83%97%E3%83%AD%E3%83%A1%E3%83%B32023"
    }

    # 辞書からデータを取り出してFlexMessageを定義
    for key, value in text_dict.items():
      temp.append(
        {
          "type": "button",
          "action": {
            "type": "uri",
            "label": key,
            "uri": value
          },
          "style": "primary"
        }
      )
    payload = {
      "type": "bubble",
      "body": {
        "type": "box",
        "layout": "vertical",
        "contents": temp
      }
    }
    return payload

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.message.text == "プロメン":
        payload = make_flex()
        flex_message = FlexSendMessage(
            alt_text='this is alt_text',
            contents=payload
          )
        line_bot_api.reply_message(event.reply_token, flex_message)

    else:
        reply_text = event.message.text  # 受信したテキストメッセージを取得
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=reply_text)  # 受信したテキストメッセージをそのまま返信
        )
        print("返信完了!!\ntext:", event.message.text)  # 返信が完了したことを表示




ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行

## 例4 カルーセル

カルーセルを使うと、複数のオブジェクトをユーザーがスクロールして閲覧することができるようになります。

カルーセルについては[公式のドキュメント：カルーセル](https://developers.line.biz/ja/docs/messaging-api/message-types/#carousel-template)などを参考にしてください。

作り方について、参考記事は[こちら](https://zanote.net/python/carouseltemplate/)です。

この記事では、カルーセルを使って多くのメッセージを表示させる対応方法についても触れられています。

以下では、[こちら](https://github.com/kiryu-3/Prmn2023)で公開されているプロメンの資料について、
カルーセルで閲覧できるようにしています。

＜実行イメージ＞

![](https://imgur.com/TLsJfuG.png)

＜実行イメージ＞

![](https://imgur.com/AZ11RML.png)

![](https://imgur.com/V1mXZBg.png)

In [ ]:
import os
from pathlib import Path
import uuid

from flask import Flask, request, abort, redirect
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ImageMessage, ImageSendMessage, PostbackEvent,
    CarouselTemplate, CarouselColumn, RichMenu, RichMenuSize, RichMenuArea, RichMenuBounds, PostbackAction, URIAction,
    QuickReply, QuickReplyButton
)
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    reply_text = event.message.text  # 受信したテキストメッセージを取得
    if reply_text == "プロメン":
        make_quick_reply(event.reply_token, "プロメンの参考資料について表示しますか？")
    else:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=reply_text)  # 受信したテキストメッセージをそのまま返信
        )
    print("返信完了!!\ntext:", event.message.text)  # 返信が完了したことを表示

# PostbackActionがあった時のアクション
@handler.add(PostbackEvent)
def on_postback(event):
    postback_msg = event.postback.data

    if postback_msg == "資料を表示":
        columns_list = []
        url_dict = {
            "Python（基本文法）" : ["プロメン2023の本資料。\nPythonの基本文法についてまとめている。", "https://kiryu-3.github.io/Prmn2023/python-basic/index.html#0"],
            "Python（WEB開発）" : ["プロメン2023の最終成果物参考資料。\nPythonで簡単にWEB開発を行えるStreamlitについてまとめている。", "https://kiryu-3.github.io/Prmn2023/streamlit-prmn/index.html#0"],
            "Python（データサイエンス）" : ["おまけ資料。\nデータサイエンスのライブラリについてまとめている。", "https://kiryu-3.github.io/Prmn2023/python-ds/index.html#0"],
            "Python（統計）" : ["おまけ資料。\n統計の基礎についてまとめている。", "https://kiryu-3.github.io/Prmn2023/python-stat/index.html#0"],
            "Python（機械学習）" : ["おまけ資料。\n機械学習の基礎についてまとめている。", "https://kiryu-3.github.io/Prmn2023/python-machine-prmn/index.html#0"],
            "フロントエンド入門" : ["おまけ資料。\nHTML&CSS&javascriptの基礎についてまとめている。", "https://kiryu-3.github.io/Prmn2023/frontend-prmn/index.html#0"],
            "データベース入門" : ["おまけ資料。\nSQLの基礎についてまとめている。", "https://kiryu-3.github.io/Prmn2023/sql-prmn/index.html#0"],
        }
        for key, value in url_dict.items():
          columns_list.append(CarouselColumn(title=key, text=value[0], actions=[URIAction(label='リンクはこちら', uri=value[1])]))

        carousel_template_message = TemplateSendMessage(
                        alt_text='会話ログを表示しています',
                        template=CarouselTemplate(columns=columns_list)
                        )
        line_bot_api.reply_message(event.reply_token, messages=carousel_template_message)

    elif postback_msg == "キャンセル":
        messages = TextSendMessage(text="キャンセルしました")
        line_bot_api.reply_message(event.reply_token, messages=messages)


def make_quick_reply(token, text):
    items = []
    items.append(QuickReplyButton(action=PostbackAction(label='表示する', data='資料を表示')))
    items.append(QuickReplyButton(action=PostbackAction(label='キャンセル', data='キャンセル')))
    messages = TextSendMessage(text=text,
                            quick_reply=QuickReply(items=items))
    line_bot_api.reply_message(token, messages=messages)

ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行


## 例5 リッチメニュー

**リッチメニュー**とは、トークルームのキーボードエリアにアカウント独自のメニューを展開できる機能です。

リッチメニューについては[公式のドキュメント：リッチメニュー](https://developers.line.biz/ja/docs/messaging-api/using-rich-menus/)などを参考にしてください。



作り方について、参考記事は[こちら](https://zanote.net/python/rich-menu/)です。

この記事では、カルーセルを作る際に必要な引数などの解説があります。

以下では、YAHOO!社の主要サイトをリッチメニューで表示し、
それぞれのサイトに飛べるようにしています。

＜実行イメージ＞

![](https://imgur.com/bH1djoH.png)

In [ ]:
import os
from pathlib import Path
import uuid

from flask import Flask, request, abort, redirect
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage, PostbackEvent
from linebot.models import (
    RichMenu, RichMenuSize, RichMenuArea, RichMenuBounds, PostbackAction, URIAction
)
from pyngrok import ngrok
import dotenv

app = Flask(__name__)
# LineBotApiオブジェクトを作成
dotenv.load_dotenv("./info/.env")
line_bot_api = LineBotApi(os.environ["CHANNEL_ACCESS_TOKEN"])
handler = WebhookHandler(os.environ["CHANNEL_SECRET"])

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)  # リクエストの署名検証を行い、正しければハンドラを実行
    except InvalidSignatureError:
        abort(400)  # 署名が無効な場合はエラーを返す
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    reply_text = event.message.text  # 受信したテキストメッセージを取得
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_text)  # 受信したテキストメッセージをそのまま返信
    )
    print("返信完了!!\ntext:", event.message.text)  # 返信が完了したことを表示

@handler.add(PostbackEvent)
def handle_postback(event):
    data = event.postback.data  # ポストバックデータを取得

    if data == 'renew':
        # renewアクションが実行された場合の処理
        # 特定のサイトのURLにリダイレクト
        redirect_url = "https://news.yahoo.co.jp/categories/domestic"
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=redirect_url)  # メッセージとしてURLを返信
        )


# リッチメニューの作成
rich_menu_to_create = RichMenu(
    size=RichMenuSize(width=2500, height=1250),
    selected=False,
    name="Nice rich menu",
    chat_bar_text="Tap to open",
    areas=[
        RichMenuArea(
            bounds=RichMenuBounds(x=0, y=0, width=1250, height=625),
            action=URIAction(label='スポーツナビ', uri='https://sports.yahoo.co.jp/')
        ),
        RichMenuArea(
            bounds=RichMenuBounds(x=1250, y=0, width=1250, height=625),
            action=URIAction(label='知恵袋', uri='https://chiebukuro.yahoo.co.jp/?fr=ytop_menu')
        ),
        RichMenuArea(
            bounds=RichMenuBounds(x=0, y=625, width=1250, height=625),
            action=URIAction(label='ニュース', uri='https://news.yahoo.co.jp/')
        ),
        RichMenuArea(
            bounds=RichMenuBounds(x=1250, y=625, width=1250, height=625),
            action=URIAction(label='天気・災害', uri='https://weather.yahoo.co.jp/weather/')
        )
    ]
)

# 1つ目のエリアの画像を設定
with open("/content/merged.png", 'rb') as f:
    rich_menu_area1_image = f.read()


# リッチメニューの作成と画像の設定
rich_menu_id = line_bot_api.create_rich_menu(rich_menu=rich_menu_to_create)
line_bot_api.set_rich_menu_image(rich_menu_id, "image/png", rich_menu_area1_image)


# リッチメニューの作成と画像の設定
line_bot_api.set_default_rich_menu(rich_menu_id)

rich_menu_list = line_bot_api.get_rich_menu_list()
print(rich_menu_list)

ngrok_tunnel = ngrok.connect(5000)  # ポート5000でngrokのトンネルを作成
print('Public URL:', ngrok_tunnel.public_url)  # 公開されたURLを表示

if __name__ == "__main__":
    app.run()  # アプリケーションを実行


### 画像の作り方

リッチメニューに使う画像の作り方（結合方法）について紹介しています。

参考記事は[こちら](https://note.nkmk.me/python-opencv-hconcat-vconcat-np-tile/)です。

[公式ドキュメント](https://developers.line.biz/ja/docs/messaging-api/using-rich-menus/#prepare-a-rich-menu-image)にもありますが、使える画像には制限がありますので注意して下さい。

#### Colabにアップロードした画像の結合

In [ ]:
import cv2
import numpy as np

# 画像のURLリスト
png_list = [
    '/content/image(0).png',
    '/content/image(1).png',
    '/content/image(2).png',
    '/content/image(3).png'
]
image_list = list()

for idx, image_path in enumerate(png_list):
    image_path = cv2.imread(f"/content/image({idx}).png")
    image_s = cv2.resize(image_path, dsize=(0, 0), fx=0.5, fy=0.5)
    image_list.append(image_s)

def concat_tile(im_list_2d):
    return cv2.vconcat([cv2.hconcat(im_list_h) for im_list_h in im_list_2d])

im_tile = concat_tile([[image_list[0], image_list[1]],
                       [image_list[2], image_list[3]]])
cv2.imwrite('/content/richtie2.png', im_tile)

# 画像の読み込み
image = cv2.imread('/content/richtie.png')

# 画像のリサイズ
resized_image = cv2.resize(image, (800, 540))

# リサイズ後の画像の保存
cv2.imwrite('/content/resized_richtile.png', resized_image)

#### imgurにアップロードした画像の結合

In [ ]:
import cv2
import numpy as np
import urllib.request

# 画像のURLリスト
url_list = [
    'https://imgur.com/YcgCkBn.png',
    'https://imgur.com/FuNYAGX.png',
    'https://imgur.com/YWWMTXq.png',
    'https://imgur.com/8AqHobb.png'
]
image_list = list()
for idx, image_url in enumerate(url_list):

    # 画像をダウンロードして保存する
    urllib.request.urlretrieve(image_url, f'image({idx}).png')

    image_path = cv2.imread(f"/content/image({idx}).png")
    image_s = cv2.resize(image_path, dsize=(0, 0), fx=0.5, fy=0.5)
    image_list.append(image_s)

def concat_tile(im_list_2d):
    return cv2.vconcat([cv2.hconcat(im_list_h) for im_list_h in im_list_2d])

im_tile = concat_tile([[image_list[0], image_list[1]],
                       [image_list[2], image_list[3]]])
cv2.imwrite('/content/richtie.png', im_tile)

# 画像の読み込み
image = cv2.imread('/content/richtie.png')

# 画像のリサイズ
resized_image = cv2.resize(image, (800, 540))

# リサイズ後の画像の保存
cv2.imwrite('/content/resized_richtile.png', resized_image)

## 最後に

応用編の資料はこれで終了です。
プロメン最終成果物の参考にしてください。
